In [1]:
import pandas as pd
import numpy as np
import torch
import h5py
import sys
import cvxpy as cp


In [ ]:
class params:
    trade_time_list = ["1129", "1429"]
    all_time_list = ['0959', '1029', '1059', '1129', '1329', '1359', '1429', "1455"]
    today_open_trade_task = pd.DataFrame()
    pt_asset_1129 = 19994192.0
    pt_asset_1429 = 20075239.14
    citic_share_limit = 0.05
    cmvg_share_limit = 0.075
    pt_tvr_limit = 15.0
    rq_tvr_limit = 40.0
    pt_max_prop_total = 2.0
    rq_max_prop_total = 5.0
    trade_retmax = 9.0
    trade_retmin = -9.0
    pt_liquid_ratio = 3.0
    rq_liquid_ratio = 3.0
    buy_prop_normal = 20.0

In [3]:
def noteq(a, b, eps=1e-8):# 不等于
    return abs(a-b) >= eps

def my_round2(x):  # 此处是因为python浮点数不精确
    return round(x + 1e-10, 2)


In [ ]:
def get_can_trade_amount(stock_list, time, score_array, params):
    # 记录这一时刻之前已经发生的多头账户买入和空头账户卖出的交易记录
    # 多头账户卖出和空头账户还券不需要记录
    # 原则上，今天已经买入的多头股票，不能再进行空头融券卖出，已经融券卖出的空头融券，不能再进行多头账户买入
    if len(params.today_open_trade_task) > 0:
        today_buy_stock_list = list(params.today_open_trade_task.loc[(params.today_open_trade_task.side == "B") & (params.today_open_trade_task.a_trans > 0) & (params.today_open_trade_task.type == "PT"), "code"].unique())
        today_sell_stock_list = list(params.today_open_trade_task.loc[(params.today_open_trade_task.side == "S") & (params.today_open_trade_task.a_trans > 0) & (params.today_open_trade_task.type == "RQ"), "code"].unique())
    else:
        today_buy_stock_list = []
        today_sell_stock_list = []

    # 同样获取当日持仓
    # 获取当前持仓股票的volume, 框架里面rq_volume全都是负数，-100代表融券100股
    stock_hold_pt = {code:round(stock_info.pt_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.pt_volume > 0}
    stock_available_pt = {code:round(stock_info.pt_available_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.pt_volume > 0}
    stock_hold_rq = {code:round(stock_info.rq_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.rq_volume < 0}
    stock_available_rq = {code:round(stock_info.rq_available_volume, 2)  for code, stock_info in params.account.stock_infos.items() if stock_info.rq_volume < 0}

    # 获取昨收价格、涨跌停价格、融券余量、实时价格，都是您那边直接获取
    today_preclose_dict = params.stock_daily_data_set[params.date]['preclose'] # 昨收价
    today_ztprice_dict = params.stock_daily_data_set[params.date]['ztprice'] # 涨停价
    today_dtprice_dict = params.stock_daily_data_set[params.date]['dtprice'] # 跌停价
    today_trade_price_dict = params.stock_trade_price_data_set[params.date] # 交易价格
    today_realtime_rq_dict = params.stock_realtime_rq_data_set[params.date] # 融券余量
    time0 = params.all_time_list[params.all_time_list.index(time)-1] # 其实就是获取30分钟前的时间点

    stock_buy_liquid = {}
    stock_sell_liquid = {}
    stock_price_now = {}

    for stock in stock_list:
        if stock not in today_preclose_dict.keys():
            continue
        # 一些强制买入卖出
        pc = today_preclose_dict[stock] 
        if pc == 0.0 or pc != pc: # 如果昨收价为0或者为空值
                continue

        # 涨跌停限制
        p0 = today_trade_price_dict[stock][rf"price_{time}"].iloc[0] # 获取当前股票价格p0
        if p0 == 0.0 or p0 != p0: # 如果当前价为0或者为空值
            stock_price_now[stock] = pc
            stock_buy_liquid[stock] = 0
            stock_sell_liquid[stock] = 0
            continue

        # 涨跌停对交易的限制
        stock_price_now[stock] = p0

        zp = today_ztprice_dict[stock] # 涨停价
        dp = today_dtprice_dict[stock] # 跌停价
        hp = my_round2(pc + (zp - pc) * params.open_params.pt.trade_retmax / 10) # 风控最高价
        lp = my_round2(pc + (pc - dp) * params.open_params.pt.trade_retmin / 10) # 风控最低价
        if p0 >= hp or p0 <= lp: # 盘口涨幅突破上下限
            stock_sell_liquid[stock] = 0
            stock_buy_liquid[stock] = 0
            continue


        # 获取这个时点预估成交金额，这里直接使用过去半个小时的成交金额,例如1129就是获取的1059.59.999-1129.59.999的成交金额累计
        es_amt = today_trade_price_dict[stock][rf'amt_{time0}_{time}'].fillna(0).iloc[0]

        # 这一部分代表盘口容量限制
        stock_buy_liquid[stock] = int(es_amt* (params.pt_liquid_ratio / 100.0)) + 1
        stock_sell_liquid[stock] = int(es_amt* (params.rq_liquid_ratio / 100.0)) + 1

        # 获取当前各个仓位，并转化为正数
        v_pt_avai = stock_available_pt.get(stock, 0) # 普通户可以卖出的持仓
        v_rq_avai =  - stock_available_rq.get(stock, 0) # 融券户可以买入的持仓
        v_pt_hold = stock_hold_pt.get(stock, 0)
        v_rq_hold = - stock_hold_rq.get(stock, 0)

        # 强制平仓条件：当天不可交易或者持仓周期过长，但如果是多头股票且该票当天本身就处于比较头部的位置，则可以不强制平仓
        # params.stock_expired_days记录了股票到期日，如果是股票多头，则默认是15天(避免同一只股票持仓周期过长)，如果是股票空头，则是实际融券到期日
        # 这里采用的方式是通过对打分进行调整，让组合优化自动识别到应该强制平仓这些股票    
        if (stock not in params.today_istrade_stock_list) or (params.stock_expired_days.get(stock, 10) <= 0):
            if v_pt_hold > 0:
                if score_array[stock] < (1 - params.buy_prop_normal / 100.0):
                    score_array[stock] = 0
                    stock_sell_liquid[stock] = min(stock_sell_liquid[stock], int(v_pt_hold * stock_price_now[stock]) + 1)
            if v_rq_hold > 0:
                score_array[stock] = 1
                stock_buy_liquid[stock] = min(stock_buy_liquid[stock], int(v_rq_hold * stock_price_now[stock]) + 1)


        # 如果今天发生过融券卖出，则现在不能普通买入, 只能还券
        if stock in today_sell_stock_list: 
            stock_buy_liquid[stock] = min(stock_buy_liquid[stock], int(v_rq_avai * p0)  + 1)

        # 如果已经有持仓，则可以卖出金额为当前可以卖出的金额 + 可以融券卖出的金额
        if stock in today_buy_stock_list: # 不能融券卖出
            can_sell_vol = v_pt_avai
        else:
            can_sell_vol = v_pt_avai + today_realtime_rq_dict.get(stock, 0)
        if can_sell_vol > 0:
            stock_sell_liquid[stock] = min(stock_sell_liquid[stock],  int(can_sell_vol * p0)  + 1)
        else:
            stock_sell_liquid[stock] = 0


    return pd.Series(stock_price_now), pd.Series(stock_buy_liquid), pd.Series(stock_sell_liquid),score_array